# R365 Sample Data Project

## Set-Up

In [1]:
import pandas as pd
import numpy as np

with open("Revops-analyst-database.csv") as f:
    df = pd.read_csv(f)

In [2]:
df.head()

,Name,Title,Company,Company Type,Lead Created,Lead Status,Converted Date,Opportunity Close Date,Oppportunity Status,Opportunity Amount
0,Person 1,Owner,Company 1,Pancake House,1-Jan,Converted,2-Jan,15-Jan,Won,"$10,000"
1,Person 2,VP of Operations,Company 2,Pizza Place,1-Jan,Converted,2-Jan,30-Jan,Won,"$8,000"
2,Person 3,Franchisee,Company 3,Fancy Dining,1-Jan,Converted,3-Jan,5-Feb,Won,"$12,000"
3,Person 4,COO,Company 4,Burger Joint,1-Jan,Converted,10-Jan,10-Feb,Lost,"$20,000"
4,Person 5,CFO,Company 5,Pizza Place,2-Jan,Converted,2-Jan,10-Jan,Won,"$4,000"


## Question 1:
### What is our win rate (%) for each job title?

In [3]:
# First convert strings to boolean values
def to_bool(val):
    if val == 'Won':
        return 1
    else:
        return 0
df["Opportunity Status Value"] = df["Oppportunity Status"].apply(to_bool)

# Group the data by title and compute the mean of groups 
title_groups = df.groupby("Title")

win_rates = title_groups.mean().sort_values(by="Opportunity Status Value",ascending=False).rename(columns={"Opportunity Status Value":"Win Rate (%)"})

### Results

In [4]:
win_rates

,Win Rate (%)
Title,
Franchisee,0.812500
VP of Operations,0.761905
COO,0.750000
Owner,0.727273
CFO,0.714286


In [5]:
for i in range(5):
    item = win_rates.iloc[i]
    perc = "{:.2%}".format(item.values[0])
    title = item.name
    print(f'The {title} Win Rate (%) is {perc}.')

The Franchisee Win Rate (%) is 81.25%.
The VP of Operations Win Rate (%) is 76.19%.
The COO Win Rate (%) is 75.00%.
The Owner Win Rate (%) is 72.73%.
The CFO Win Rate (%) is 71.43%.


# ----------

## Question 2:
### You’re meeting with the head of Marketing. 
### Which job title and business type would you recommend they prioritize in their outreach, and why?

In [6]:
# Convert $ amounts to integer values.
def to_int(val):
    strip_val = val.replace('$','').replace(',','')
    return int(strip_val)
def rnd(val):
    return np.round(val,2)

df["Opportunity Integer Amount"] = df["Opportunity Amount"].apply(to_int)

multi_group_sum = df.groupby(['Company Type','Title']).sum()
multi_group_avg = df.groupby(['Company Type','Title']).mean().apply(rnd)
multi_group_avg["Opportunity Amount Total"] = multi_group_sum["Opportunity Integer Amount"]

In [7]:
multi_group_avg

Opportunity Status Value  \
Company Type  Title                                        
Burger Joint  CFO                                   0.25   
              COO                                   0.38   
              Franchisee                            0.00   
              Owner                                 0.00   
              VP of Operations                      0.00   
Fancy Dining  CFO                                   1.00   
              COO                                   1.00   
              Franchisee                            1.00   
              Owner                                 1.00   
              VP of Operations                      1.00   
Pancake House CFO                                   1.00   
              COO                                   1.00   
              Franchisee                            1.00   
              Owner                                 1.00   
              VP of Operations                      1.00   
Pizza Place   CFO                                   1.00   
              COO                                   1.00   
              Franchisee                            1.00   
              Owner                                 1.00   
              VP of Operations                      1.00   

                                Opportunity Integer Amount  \
Company Type  Title                                          
Burger Joint  CFO                                  9812.50   
              COO                                  5325.00   
              Franchisee                           4833.33   
              Owner                                8500.00   
              VP of Operations                     5000.00   
Fancy Dining  CFO                                 14600.00   
              COO                                 19500.00   
              Franchisee                           9437.50   
              Owner                               10800.00   
              VP of Operations                    13500.00   
Pancake House CFO                                  7300.00   
              COO                                  5716.67   
              Franchisee                           2200.00   
              Owner                                9185.71   
              VP of Operations                     6392.86   
Pizza Place   CFO                                  6500.00   
              COO                                  4333.33   
              Franchisee                           8200.00   
              Owner                                3625.00   
              VP of Operations                     6285.71   

                                Opportunity Amount Total  
Company Type  Title                                       
Burger Joint  CFO                                  78500  
              COO                                  42600  
              Franchisee                           14500  
              Owner                                51000  
              VP of Operations                     25000  
Fancy Dining  CFO                                  73000  
              COO                                  58500  
              Franchisee                           75500  
              Owner                                54000  
              VP of Operations                     27000  
Pancake House CFO                                  36500  
              COO                                  34300  
              Franchisee                            4400  
              Owner                                64300  
              VP of Operations                     44750  
Pizza Place   CFO                                  19500  
              COO                                  13000  
              Franchisee                           24600  
              Owner                                14500  
              VP of Operations                     44000

### Burger Joints are the only type of restaurant that we've lost opportunities with.
### So, let's decide the which job titles Marketing should target first.

In [8]:
# Group the burger joint rows by job title and opportunity status
# Then count the opportunities, and sum/average the amounts.
burger_group = df.groupby(['Company Type']).get_group('Burger Joint').groupby(['Title','Oppportunity Status'])
counts = burger_group.count()['Name'].values
burger_avg = burger_group.mean().apply(rnd).rename(columns={"Opportunity Integer Amount":"Opportunity Amount Average"})
burger_avg["Opportunity Count"] = counts
burger_avg["Opportunity Amount Total"] = burger_group.sum().apply(rnd)["Opportunity Integer Amount"]
burger_avg = burger_avg[["Opportunity Count","Opportunity Amount Average","Opportunity Amount Total"]]

In [9]:
burger_avg

Opportunity Count  \
Title            Oppportunity Status                      
CFO              Lost                                 6   
                 Won                                  2   
COO              Lost                                 5   
                 Won                                  3   
Franchisee       Lost                                 3   
Owner            Lost                                 6   
VP of Operations Lost                                 5   

                                      Opportunity Amount Average  \
Title            Oppportunity Status                               
CFO              Lost                                   11666.67   
                 Won                                     4250.00   
COO              Lost                                    7540.00   
                 Won                                     1633.33   
Franchisee       Lost                                    4833.33   
Owner            Lost                                    8500.00   
VP of Operations Lost                                    5000.00   

                                      Opportunity Amount Total  
Title            Oppportunity Status                            
CFO              Lost                                    70000  
                 Won                                      8500  
COO              Lost                                    37700  
                 Won                                      4900  
Franchisee       Lost                                    14500  
Owner            Lost                                    51000  
VP of Operations Lost                                    25000

### CFOs and COOs are the titles with the most opportunities and the only two titles that we've had some traction with.
### As far as potential amount goes, losing opportunities with CFOs results in the largest amount missed:
#### $11,666.67 on average
### CFOs are followed by Owners:
#### $8,500 on average
### Then COOs:
#### $7,540 on average
### If we factor in the amount gained from the CFO/COO wins, losing opportunities with Owners is the most costly loss, around $1,000 above CFO losses.
### Therefore, I would recommend Marketing prioritize Burger Joint CFOs, as there appear to be more lucrative opportunaties with CFOs, and we have had some previous success with CFOs. After CFOs, I would recommend they target Burger Joint Owners because there are quite a few lucrative opportunities that we have yet to have success with.
# ----------


## Question 3:
### If a lead converts the same day, what is the probability that the deal will be won?

### As a lead will either be won or lost, the data follows a Binomial distribution, with n = 43. Given this sample size, I'm going to use Wilson's method to get a confidence interval for the probability.

In [10]:
import scipy.stats as st

In [11]:
same_day = df.loc[df['Lead Created'] == df['Converted Date']]
data = same_day['Opportunity Status Value']
k = np.sum(data)
n = data.shape[0]
p = np.mean(data)
print(f'There are {n} total observations with {k} wins.')
print(f'The sample mean is {rnd(p)} and the sample standard deviation is {rnd(np.std(data))}.')

There are 43 total observations with 38 wins.
The sample mean is 0.88 and the sample standard deviation is 0.32.


In [12]:
# Perform a binomial test on the data.
result = st.binomtest(k,n=n,p=p)
estp = result.proportion_estimate
conf = result.proportion_ci(confidence_level=0.95)
low = "{:.2%}".format(conf[0])
high = "{:.2%}".format(conf[1])

print(f'If a lead converts on the same day it is created, the probability that the deal will be won is between {low} and {high}, with 95% confidence, and is estimated to be {"{:.2%}".format(estp)}.')

If a lead converts on the same day it is created, the probability that the deal will be won is between 74.92% and 96.11%, with 95% confidence, and is estimated to be 88.37%.


## Other Observations

In [13]:
from datetime import datetime

In [14]:
def to_datetime(string):
    return datetime.strptime(string,"%d-%b")

In [15]:
df['Created (DT)'] = df["Lead Created"].apply(to_datetime)
df['Converted (DT)'] = df["Converted Date"].apply(to_datetime)
df['Closed (DT)'] = df["Opportunity Close Date"].apply(to_datetime)

In [16]:
def grab_int(days):
    return days.days
df['Convert Time (Days)'] = (df['Converted (DT)'] - df['Created (DT)']).apply(grab_int)
df['Days Between Conversion and Closing'] = (df['Closed (DT)'] - df['Converted (DT)']).apply(grab_int)

In [17]:
df.describe()

,Opportunity Status Value,Opportunity Integer Amount,Convert Time (Days),Days Between Conversion and Closing
count,100.000000,100.000000,100.000000,100.00000
mean,0.750000,7994.500000,2.700000,50.73000
std,0.435194,7823.868863,3.255144,31.28683
min,0.000000,1000.000000,0.000000,1.00000
25%,0.750000,3500.000000,0.000000,24.75000
50%,1.000000,6000.000000,1.000000,42.00000
75%,1.000000,9250.000000,5.000000,74.00000
max,1.000000,50000.000000,13.000000,119.00000


In [18]:
convert_bins = [-1,0,2,6,14]
convert_labels = ['0','1-2','3-6','7+']
closing_bins = [0,25,45,75,120]
closing_labels = ['1-25','26-45','46-75','76+']
df['Conversion Time Range'] = pd.cut(df['Convert Time (Days)'],bins=convert_bins,labels=convert_labels)
df['Closing Time Range'] = pd.cut(df['Days Between Conversion and Closing'],bins=closing_bins,labels=closing_labels)

In [19]:
convert_groups = df.groupby(['Conversion Time Range']).mean()[['Opportunity Status Value','Opportunity Integer Amount']]
closing_groups = df.groupby(['Closing Time Range']).mean()[['Opportunity Status Value','Opportunity Integer Amount']]

In [20]:
convert_groups

,Opportunity Status Value,Opportunity Integer Amount
Conversion Time Range,,
0,0.883721,7598.837209
1-2,0.894737,6052.631579
3-6,0.636364,9954.545455
7+,0.375000,8668.750000


### When using the above partition, it appears we are most likely to win an opportunity if the lead converts between 0 and 2 days after creation, with 1 to 2 days conversion time being slightly more likely to result in a win than same-day conversion. 
### After 2 days without conversion, the probability that the lead will result in a win drops significantly, and past 7 days even further. 
### Furthermore, these leads that take longer to convert appear to have greater amounts on average than leads that convert in 0-2 days.
### Given these findings, more focus should be placed on a potential lead in the first 2 days after creation.

In [21]:
closing_groups

,Opportunity Status Value,Opportunity Integer Amount
Closing Time Range,,
1-25,0.730769,5761.538462
26-45,0.862069,8810.344828
46-75,0.727273,8077.272727
76+,0.652174,9410.869565


### When using the above partition, it appears that we are significantly more likely to have a win when the closing date is between 26-45 days after conversion, with these leads having the second-highest dollar amounts.
### Leads that have 76+ days between conversion and closing have the highest dollar amounts and are also the least likely to result in a win. 
### However, the difference is not as large as the results above. Leads that take longer than 75 days to close after conversion still result in a win more than half of the time.
### Given these findings, it may be beneficial to focus more energy on leads as they approach 45 days after conversion.